<a href="https://colab.research.google.com/github/FmasterofU/FmasterofU/blob/master/prepare_image_for_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import ee

In [44]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=TMF5zw8hzv2txzUsTqwgYQh4ruCBmNmyUT6RnOdGDQs&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhNY78MfLFQuO3dA0wLKp6He5AlC0zKUsx-p1ZqKfFzfA9TZus_xpU

Successfully saved authorization token.


In [45]:
import folium
from folium import plugins

In [46]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [47]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [49]:
import time
from google.colab import drive
drive.mount('/content/gdrive', True)

def exportImage(image: ee.Image, format: str, geometry: ee.Geometry, fileName: str):
  projection = image.projection().getInfo()
  task = ee.batch.Export.image.toDrive(**{
    'region': geometry.getInfo()['coordinates'],
    'fileFormat' : format,
    'folder': 'FireMaps',
    'fileNamePrefix': fileName,
    'image': image,
    'description': fileName,
    'crs': projection['crs'],
    'crsTransform': projection['transform'],
    'formatOptions': {
      'patchDimensions': [10, 10],
      #'maxFileSize': 104857600,
      'compressed': False,
    },
  })

  task.start()
  while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(10)
  print(task.status())

Mounted at /content/gdrive


In [50]:
dataset = ee.ImageCollection('FIRMS').filter(ee.Filter.date('2018-07-01', '2018-07-15')).first()
fires = dataset.select('T21');
dataset9 = ee.ImageCollection('FIRMS').filter(ee.Filter.date('2018-07-22', '2018-07-23')).first()
fires9 = dataset.select('T21');

In [51]:
def square_clip_and_scale(image: ee.Image, center_point_longitude, center_point_latitude, half_edge_clip_squarelength, px_width, px_height):
  point = ee.Geometry.Point([center_point_longitude, center_point_latitude])
  square = point.buffer(half_edge_clip_squarelength).bounds()
  return image.clipToBoundsAndScale(square, px_width, px_height), square, point

In [52]:
forestDataset = ee.Image('UMD/hansen/global_forest_change_2020_v1_8')
forests = forestDataset.select('treecover2000')
mask = forests.gt(0) # possibly change mask bound

In [54]:
firesVis = {
  'min': 325.0,
  'max': 400.0,
  'palette': ['red', 'orange', 'yellow'],
}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)

#fires1, square = square_clip_and_scale(fires, 24.21, 42.18, 15000, 10, 10)
fires1, square, point = square_clip_and_scale(fires, 26.95, -9.04, 25000, 75, 75)
maskedFires = ee.ImageCollection.fromImages([fires]).mosaic().updateMask(mask)

# Add the elevation model to the map object.
my_map.add_ee_layer(maskedFires, firesVis, 'Fires')
my_map.add_ee_layer(fires1, firesVis, 'Firesclipnscale')
my_map.add_ee_layer(square, {'color': 'blue'}, 'Single polygon (scale)')
my_map.add_ee_layer(point, {'color': 'gray'}, 'Tacka')
my_map.add_ee_layer(fires9, firesVis, 'Fires7')

my_map.add_child(folium.LayerControl())
folium.LatLngPopup().add_to(my_map)
plugins.Fullscreen().add_to(my_map)

exportImage(fires1, 'TFRecord', square, 'tfr')

display(my_map)

Polling for task (id: BMJCJWXGNX6PSOP7IYIW5DAI).
Polling for task (id: BMJCJWXGNX6PSOP7IYIW5DAI).
Polling for task (id: BMJCJWXGNX6PSOP7IYIW5DAI).
Polling for task (id: BMJCJWXGNX6PSOP7IYIW5DAI).
Polling for task (id: BMJCJWXGNX6PSOP7IYIW5DAI).
Polling for task (id: BMJCJWXGNX6PSOP7IYIW5DAI).
Polling for task (id: BMJCJWXGNX6PSOP7IYIW5DAI).
Polling for task (id: BMJCJWXGNX6PSOP7IYIW5DAI).
Polling for task (id: BMJCJWXGNX6PSOP7IYIW5DAI).
Polling for task (id: BMJCJWXGNX6PSOP7IYIW5DAI).
Polling for task (id: BMJCJWXGNX6PSOP7IYIW5DAI).
Polling for task (id: BMJCJWXGNX6PSOP7IYIW5DAI).
{'state': 'COMPLETED', 'description': 'tfr', 'creation_timestamp_ms': 1645380243984, 'update_timestamp_ms': 1645380357461, 'start_timestamp_ms': 1645380306458, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/11lzX7Fg9k0neRriPVMC19YQT-LttQN6K', 'https://drive.google.com/#folders/11lzX7Fg9k0neRriPVMC19YQT-LttQN6K'], 'attempt': 1, 'id': 'BMJCJWXGNX6PSOP7IYIW5DAI', 'name': 'pr